In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def get_masked_input_and_labels(encoded_texts, n_cat):
    # For each sentence, mask each word one-by-one

    encoded_texts_masked = []
    y_labels = []

    for encoded_text in encoded_texts:
        for i in range(len(encoded_text)):
            encoded_text_masked = np.copy(encoded_text)
            y_label = encoded_text_masked[i]
            encoded_texts_masked.append(np.delete(encoded_text_masked, i))
            y_labels.append(np.array([y_label]))

    return np.array(encoded_texts_masked), np.array(y_labels)

In [5]:
#### K = number of countries = number of capitals
#### S = number of words used by both topics
#### L = sentence length
#### q1, q2 = probability of having 1 or 2 pairs
#### embed_dim = dimension of embeddings
#### n_sentences = number of training sentences

def train_model(K, S, L, q1, q2, embed_dim, n_sentences):
    
    countries = ['country_' + str(i) for i in range(K)]
    capitals = ['capital_' + str(i) for i in range(K)]
    randoms = ['random_' + str(i) for i in range(S)]

    vocabs = countries + capitals + randoms
    vocab_map = {}

    for i in range(len(vocabs)):
        vocab_map[vocabs[i]] = i
        
    sentences = []
    sentences_number = []
    
    q0 = 1 - q1 - q2

    for i in range(n_sentences):

        sentence = []

        temp2 = npr.uniform()
        
        if temp2 <= q0:
            n_pairs = 0
        elif temp2 <= q0 + q1:
            n_pairs = 1
        else:
            n_pairs = 2
        
        pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
        for pair in pairs:
            temp3 = npr.uniform()
            
            if temp3 <= 0.5:
                sentence.append(countries[pair])
                sentence.append(capitals[pair])
            elif temp3 <= 0.75:
                sentence.append(countries[pair])
            else:
                sentence.append(capitals[pair])
                
        sentence += list(np.random.choice(randoms, L - len(sentence), replace = False))

        sentence_number = [vocab_map[i] for i in sentence]
        sentences.append(sentence)
        sentences_number.append(sentence_number)
        
    x_train = np.array(sentences_number)
    n_cat = len(vocab_map)
    x_masked_train, y_masked_labels_train = get_masked_input_and_labels(x_train, n_cat)
    
    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(n_cat, embed_dim, name="word_embedding")(inputs)
    encoder_output = layers.GlobalAveragePooling1D()(word_embeddings)
    mlm_output = layers.Dense(n_cat, name="mlm_cls", activation="softmax", use_bias=False)(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=500, batch_size=128, verbose=0)
    
    return sentences, vocab_map, mlm_model

In [6]:
def get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples):
    
    sentences, vocab_map, current_model = train_model(K, S, L, q1, q2, embed_dim, n_sentences)

    acc_countries = []
    prob_countries = []

    for _ in range(n_samples):
        sentence = []
        random_countries = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_country in random_countries:
            sentence.append('country_' + str(random_country))
            sentence.append('capital_' + str(random_country))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['capital_' + str(random_countries[-1])]
        acc_countries.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_countries.append(temp[0][vocab_map['capital_' + str(random_countries[-1])]])
        

    return sentences, current_model, vocab_map, (np.mean(acc_countries), np.mean(prob_countries))

In [7]:
K = 10 # number of countries
L = 8 # sentence length
S = 20 # number of words used by both topics
embed_dim = 100 # CBOW embedding dimension
n_sentences = 50000 # number of sentences in the training set
n_samples = 1000

In [9]:
q0 = 0 # probability of having 0 pairs
q1 = 1 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(0.0, 1.27834e-28)
(0.0, 4.444101e-31)
(0.0, 4.658759e-30)
(0.0, 1.4080529e-30)
(0.0, 5.567694e-30)
(0.0, 1.9125569e-29)
(0.0, 4.301042e-28)
(0.0, 5.2385323e-31)
(0.0, 4.0240155e-30)
(0.0, 3.1978114e-30)
(0.0, 5.968883747866692e-29)


In [10]:
q0 = 0 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(0.0, 0.001392188)
(0.0, 0.0014087613)
(0.0, 0.0013881044)
(0.0, 0.0015317573)
(0.0, 0.001432816)
(0.0, 0.0017275527)
(0.0, 0.0013761909)
(0.0, 0.0014950656)
(0.0, 0.0012925585)
(0.0, 0.0015959908)
(0.0, 0.0014640985638834534)


In [11]:
q0 = 1/2 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(0.0, 9.229606e-10)
(0.0, 6.7816317e-09)
(0.0, 6.5903984e-09)
(0.0, 1.4319855e-08)
(0.0, 3.4659073e-09)
(0.0, 1.0837894e-09)
(0.0, 2.7575278e-09)
(0.0, 2.4959612e-09)
(0.0, 1.7927567e-08)
(0.0, 8.553309e-09)
(0.0, 6.489890724825641e-09)


In [12]:
q0 = 1/2 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(1.0, 0.49655265)
(1.0, 0.5201077)
(1.0, 0.49363825)
(1.0, 0.4974702)
(1.0, 0.5078312)
(1.0, 0.5074527)
(1.0, 0.48843002)
(1.0, 0.5032265)
(1.0, 0.5153655)
(1.0, 0.49349388)
(0.9999999999999999, 0.5023568630218506)


In [13]:
q0 = 0 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(0.0, 0.030038323)
(0.0, 0.032365248)
(0.081, 0.040069006)
(0.0, 0.029754665)
(0.009, 0.033585098)
(0.011, 0.034003943)
(0.015, 0.034744516)
(0.001, 0.030264061)
(0.004, 0.028253706)
(0.004, 0.032905467)
(0.012499999999999997, 0.03259840328246355)


In [14]:
q0 = 1/3 # probability of having 0 pairs
q1 = 1/3 # probability of having 1 pair
q2 = 1/3 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(1.0, 0.26847592)
(1.0, 0.26693115)
(1.0, 0.265935)
(1.0, 0.24719693)
(1.0, 0.2652851)
(1.0, 0.25266227)
(1.0, 0.28976962)
(1.0, 0.27169693)
(1.0, 0.2785178)
(1.0, 0.2963298)
(0.9999999999999999, 0.2702800527215004)
